In [ ]:
#default_exp logargs

In [ ]:
#export
from fastcore.imports import *
from fastcore.foundation import *
from fastcore.meta import *
from functools import wraps
import inspect

In [ ]:
from fastcore.test import *
from nbdev.showdoc import *
from fastcore.nb_imports import *

# log_args

> Decorator to log function args in 'to.init_args'

Note that this module may be removed in a future version. Use `store_attr` instead where possible.

In [ ]:
#export
def log_args(f=None, *, to_return=False, but=None, but_as=None):
    "Decorator to log function args in 'to.init_args'"
    if f is None: return partial(log_args, to_return=to_return, but=but, but_as=but_as)

    if inspect.isclass(f):
        f.__init__ = log_args(f.__init__, to_return=to_return, but=but, but_as=but_as)
        return f

    but_as_args = L(getattr(b, '_log_args_but', None) for b in L(but_as)).concat()
    but = (L(but.split(',') if but else None) + but_as_args + L('self')).unique()
    but_not_found = L(b for b in L(but_as) if not hasattr(b, '_log_args_but'))
    if but_not_found: print(f'Did not find but_as with {f.__qualname__} in {[b.__qualname__ for b in but_not_found]}')
    setattr(f, '_log_args_but', but)

    @wraps(f)  # maintain original signature
    def _f(*args, **kwargs):
        f_insp,args_insp = f,args
        xtra_kwargs = {}
        # some functions don't have correct signature (e.g. functions with @delegates such as Datasets.__init__)
        if '__init__' in f.__qualname__:
            # from https://stackoverflow.com/a/25959545/3474490
            # args[0].__class__ would not consider inheritance
            cls = getattr(inspect.getmodule(f), f.__qualname__.split('.<locals>', 1)[0].rsplit('.', 1)[0])
            f_insp, args_insp = cls, args[1:]
        try: func_args = inspect.signature(f_insp).bind(*args_insp, **kwargs)
        except Exception as e:
            try:
                # sometimes it happens because the signature does not reference some kwargs
                sigp = dict(inspect.signature(f_insp).parameters)
                key_no_sig = set(kwargs.keys())-set(sigp.keys())
                #if key_no_sig: print(f'Warning: @log_args found unexpected args in {f.__qualname__}: {key_no_sig}')
                xtra_kwargs={k:kwargs.pop(k) for k in key_no_sig}
                func_args = inspect.signature(f_insp).bind(*args_insp, **kwargs)
            except:
                #print(f'@log_args had an issue on {f.__qualname__} -> {e}')
                return f(*args, **kwargs)
        func_args.apply_defaults()
        log_dict = {**func_args.arguments, **{f'{k} (not in signature)':v for k,v in xtra_kwargs.items()}}
        log = {f'{f.__qualname__}.{k}':v for k,v in log_dict.items() if k not in but}
        inst = f(*args, **kwargs) if to_return else args[0]
        init_args = getattr(inst, 'init_args', {})
        init_args.update(log)
        setattr(inst, 'init_args', init_args)
        return inst if to_return else f(*args, **kwargs)
    return _f

In [ ]:
class tst:
    @log_args
    def __init__(self, a, b, c=3, d=4):
        pass
test_eq(tst(1,2).init_args, {f'tst.__init__.{k}':v for k,v in dict(a=1,b=2,c=3,d=4).items()})

Use `log_args` to save function args in `to.init_args`. Optional args are:

* `to_return`: applies to return value if True (for functions), otherwise to `self` (for class instances)
* `but`: args that we do not want to save separated by ','
* `but_as`: pull `but` arg from another `log_args` (which cannot have used `to_return=True`)

Notes:

* `@log_args` needs to be placed below `@patch` and above `@funcs_kwargs` or `@delegates`
* when wrapping a class, it will wrap its `__init__` method

In [ ]:
class tst:
    @log_args
    def __init__(self, a, b, c=3, d=4): pass
test_eq(tst(1,2).init_args, {f'tst.__init__.{k}':v for k,v in dict(a=1,b=2,c=3,d=4).items()})

@log_args
class tst:
    def __init__(self, a, b, c=3, d=4): pass
test_eq(tst(1,2).init_args, {f'tst.__init__.{k}':v for k,v in dict(a=1,b=2,c=3,d=4).items()})

@log_args(but='a,c')
class tst:
    def __init__(self, a, b, c=3, d=4): pass
test_eq(tst(1,2).init_args, {f'tst.__init__.{k}':v for k,v in dict(b=2,d=4).items()})

class tst: pass
@log_args(to_return=True)
def tst_f(a,b): return tst
test_eq(tst_f(1,2).init_args, {f'tst_f.{k}':v for k,v in dict(a=1,b=2).items()})

@log_args
@funcs_kwargs
class tst:
    _methods='a'.split()    
    def __init__(self, **kwargs): pass
test_eq(tst(a=noop).init_args['tst.__init__.a'], noop)

In [ ]:
class tst_base:
    def __init__(self, a=None): pass

@log_args
@delegates(tst_base)
class tst:
    def __init__(self, **kwargs): pass
test_eq(tst(a=1).init_args['tst.__init__.a'], 1)

@log_args
class tst_parent:
    def __init__(self, a): pass

In [ ]:
@log_args
class tst(tst_parent):
    def __init__(self, b): super().__init__(a=b)
test_eq(tst(1).init_args, {'tst_parent.__init__.a': 1, 'tst.__init__.b': 1})

class tst_ref:
    def __init__(self, a, b, c, d): pass

class tst:
    def __init__(self, a=1, b=2, c=3, d=4): pass
test_stdout(lambda: log_args(tst, but_as=tst_ref.__init__),
            "Did not find but_as with tst.__init__ in ['tst_ref.__init__']")

In [ ]:
@log_args(but='a,b')
class tst_ref:
    def __init__(self, a, b, c, d): pass

@log_args(but_as=tst_ref.__init__)
class tst:
    def __init__(self, a=1, b=2, c=3, d=4):
        pass
test_eq(tst().init_args, {'tst.__init__.c': 3, 'tst.__init__.d': 4})

@log_args(but='a,b')
class tst_ref:
    def __init__(self, a, b, c, d): pass

@log_args(but='c', but_as=tst_ref.__init__)
class tst:
    def __init__(self, a=1, b=2, c=3, d=4): pass
test_eq(tst().init_args, {'tst.__init__.d': 4})

# Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_test.ipynb.
Converted 01_foundation.ipynb.
Converted 02_utils.ipynb.
Converted 03_dispatch.ipynb.
Converted 04_transform.ipynb.
Converted 05_logargs.ipynb.
Converted 06_meta.ipynb.
Converted index.ipynb.
